In [18]:
import json
import pandas as pd

In [19]:
INPUT_TSV_FILENAME = "../test_data/kelly/Covian_PairedData_TestSheet.txt"
INPUT_JSON_FILENAME = "../test_data/kelly/Covian_paired_datarecord_preview_id.json"

INSTRUMENT_METHOD_LABEL = "Seed Grant TOF"
EXTRACTION_METHOD = "SeedGrant_Extraction"
SAMPLE_PREP = "metagenomic_mouse_fecal"

In [20]:
pairing_df = pd.read_csv(INPUT_TSV_FILENAME, sep=None)
pairing_df.head()

/home/mingxun/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Metabolomics_ftp experiment_accession
0  ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...           ERS4356233
1  ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...           ERS4356234
2  ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...           ERS4356235
3  ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...           ERS4356236
4  ftp://massive.ucsd.edu/MSV000083302/peak/mzxml...           ERS4356237

In [21]:
template_json = json.loads(open(INPUT_JSON_FILENAME).read())

In [22]:
# Preparing the genome tables

all_genome_links = []
for pair in pairing_df.to_dict(orient="records"):
    genome_dict = {}
    genome_dict["genome_label"] = pair["experiment_accession"]
    genome_dict["genome_ID"] = {
        "genome_type" : "metagenome",
        "ENA_NCBI_accession" : pair["experiment_accession"]
    }
    genome_dict["BioSample_accession"] = pair["experiment_accession"]
    all_genome_links.append(genome_dict)

In [23]:
# Preparing the join table

all_paired_links = []
for pair in pairing_df.to_dict(orient="records"):
    link_dict = {}
    link_dict["genome_label"] = pair["experiment_accession"]
    link_dict["metabolomics_file"] = pair["Metabolomics_ftp"]
    link_dict["sample_preparation_label"] = SAMPLE_PREP
    link_dict["extraction_method_label"] = EXTRACTION_METHOD
    link_dict["instrumentation_method_label"] = INSTRUMENT_METHOD_LABEL
    
    all_paired_links.append(link_dict)
    

In [24]:
# Merging it all together

template_json["genomes"] = all_genome_links
template_json["genome_metabolome_links"] = all_paired_links
with open("Kelly_formatted_json.json", "w") as output_file:
    output_file.write(json.dumps(template_json))